# ***TCGA数据挖掘***
----
> **数据下载**

In [1]:
rm(list = ls())
options(stringsAsFactors = FALSE)
getwd()

[1] "/mnt/Data/Project/TCGA"

In [3]:
### Load packages
library(dplyr)
library(tidyverse)
library(stringr)
library(data.table)
library(vroom) # vroom可以快速读取tsv文件
library(jsonlite) # 读取metadata文件

## **Data download**

In [4]:
########==== 应用R包TCGAbiolinks下载和读取TCGA数据 ====########
## 使用TCGAbiolinks处理数据，常规需要3步走，分别是检索、下载和读取数据，依次对应以下3个函数 GDCquery()、GDCdownload() 和 GDCprepare() 。
#if (!requireNamespace("BiocManager", quietly=TRUE))
#  install.packages("BiocManager")
# BiocManager::install("TCGAbiolinks", force = TRUE) # 若未安装最新的版本，请选中运行上行代码
library(TCGAbiolinks)

In [5]:
packageVersion("TCGAbiolinks")

[1] ‘2.30.4’

In [6]:
# 获取全部的TCGA项目信息
AllProj <- getGDCprojects() 

In [7]:
TCGAbiolinks:::getGDCprojects()$project_id %>% length()

[1] 86

In [9]:
projects = TCGAbiolinks:::getGDCprojects()$project_id

In [10]:
projects

[1] "TARGET-AML"                "MATCH-Z1I"                
 [3] "HCMI-CMDC"                 "MATCH-W"                  
 [5] "MATCH-Z1D"                 "MATCH-Z1A"                
 [7] "MATCH-U"                   "MATCH-Q"                  
 [9] "TCGA-PCPG"                 "MATCH-H"                  
[11] "MATCH-C1"                  "TCGA-THYM"                
[13] "MATCH-I"                   "MATCH-S1"                 
[15] "MATCH-P"                   "MATCH-R"                  
[17] "MATCH-Z1B"                 "TCGA-PAAD"                
[19] "TCGA-STAD"                 "TCGA-TGCT"                
[21] "MATCH-S2"                  "TCGA-SARC"                
[23] "TCGA-PRAD"                 "TCGA-READ"                
[25] "TCGA-UCS"                  "TCGA-UVM"                 
[27] "TRIO-CRU"                  "VAREPOP-APOLLO"           
[29] "WCDT-MCRPC"                "TARGET-ALL-P1"            
[31] "REBC-THYR"                 "TARGET-ALL-P2"            
[33] "TARGET-ALL-P3"             "ORGANOID-PANCREATIC"      
[35] "CTSP-DLBCL1"               "EXCEPTIONAL_RESPONDERS-ER"
[37] "NCICCR-DLBCL"              "MP2PRT-ALL"               
[39] "MP2PRT-WT"                 "OHSU-CNL"                 
[41] "TCGA-KICH"                 "TCGA-HNSC"                
[43] "TCGA-LUAD"                 "TCGA-LIHC"                
[45] "TCGA-LUSC"                 "TCGA-MESO"                
[47] "TCGA-LAML"                 "TCGA-LGG"                 
[49] "TCGA-KIRP"                 "TCGA-KIRC"                
[51] "BEATAML1.0-COHORT"         "CGCI-BLGSP"               
[53] "CGCI-HTMCP-CC"             "APOLLO-LUAD"              
[55] "CMI-ASC"                   "CDDP_EAGLE-1"             
[57] "CMI-MBC"                   "CGCI-HTMCP-DLBCL"         
[59] "CMI-MPC"                   "BEATAML1.0-CRENOLANIB"    
[61] "CGCI-HTMCP-LC"             "TCGA-ACC"                 
[63] "TARGET-NBL"                "TARGET-WT"                
[65] "TCGA-BLCA"                 "TARGET-OS"                
[67] "TARGET-RT"                 "FM-AD"                    
[69] "TCGA-DLBC"                 "TCGA-CHOL"                
[71] "TCGA-CESC"                 "TCGA-COAD"                
[73] "CPTAC-3"                   "TCGA-BRCA"                
[75] "TCGA-ESCA"                 "TCGA-GBM"                 
[77] "TARGET-CCSK"               "CPTAC-2"                  
[79] "MMRF-COMMPASS"             "TCGA-OV"                  
[81] "TCGA-THCA"                 "MATCH-N"                  
[83] "MATCH-B"                   "MATCH-Y"                  
[85] "TCGA-SKCM"                 "TCGA-UCEC"

In [8]:
TCGAs = grep("TCGA", projects, value = T)
sort(TCGAs)

[1] "TCGA-ACC"  "TCGA-BLCA" "TCGA-BRCA" "TCGA-CESC" "TCGA-CHOL" "TCGA-COAD"
 [7] "TCGA-DLBC" "TCGA-ESCA" "TCGA-GBM"  "TCGA-HNSC" "TCGA-KICH" "TCGA-KIRC"
[13] "TCGA-KIRP" "TCGA-LAML" "TCGA-LGG"  "TCGA-LIHC" "TCGA-LUAD" "TCGA-LUSC"
[19] "TCGA-MESO" "TCGA-OV"   "TCGA-PAAD" "TCGA-PCPG" "TCGA-PRAD" "TCGA-READ"
[25] "TCGA-SARC" "TCGA-SKCM" "TCGA-STAD" "TCGA-TGCT" "TCGA-THCA" "TCGA-THYM"
[31] "TCGA-UCEC" "TCGA-UCS"  "TCGA-UVM"

In [11]:
CPTAC = grep("CPTAC", projects, value = T)
sort(CPTAC)

[1] "CPTAC-2" "CPTAC-3"

In [12]:
TARGET = grep("TARGET", projects, value = T)
sort(TARGET)

[1] "TARGET-ALL-P1" "TARGET-ALL-P2" "TARGET-ALL-P3" "TARGET-AML"   
[5] "TARGET-CCSK"   "TARGET-NBL"    "TARGET-OS"     "TARGET-RT"    
[9] "TARGET-WT"

### **a. 检索**

In [10]:
project <-  "TARGET-OS"
# 每个项目的摘要
proj <- getProjectSummary(project)
proj

,file_count,case_count,data_category
,<int>,<int>,<chr>
1,2331,142,Simple Nucleotide Variation
2,791,263,Sequencing Reads
3,43,31,Combined Nucleotide Variation
4,2,383,Biospecimen
5,4,293,Clinical
6,333,88,Copy Number Variation
7,176,88,Transcriptome Profiling
8,258,86,DNA Methylation
9,176,88,Structural Variation


In [7]:
##(1) GDC harmonized database【GRCh38 (hg38)】
TCGAbiolinks:::getProjectSummary(project)$data_categories$data_category

[1] "Simple Nucleotide Variation"   "Sequencing Reads"             
[3] "Combined Nucleotide Variation" "Biospecimen"                  
[5] "Clinical"                      "Copy Number Variation"        
[7] "Transcriptome Profiling"       "DNA Methylation"              
[9] "Structural Variation"

In [11]:
##转录组数据的Data.category是Transcriptome Profiling，Data.type是Gene Expression Quantification，WorkflowType有四种：STAR - Counts、HTSeq - FPKM-UQ、HTSeq - FPKM和HTSeq - Counts。
Category <- "Transcriptome Profiling"
Type <- "Gene Expression Quantification"
Workflow <- "STAR - Counts"

In [12]:
# 1 - 检索需要下载的数据。GDCquery()可以通过多个参数检索限定需要下载的数据，各参数的详细说明可参阅帮助文档。
query <- GDCquery(
    project = project,
    data.category = Category,
    data.type = Type,
    workflow.type = Workflow
    )

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TARGET-OS

--------------------

oo Filtering results

--------------------

ooo By data.type

ooo By workflow.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------



### **b. 下载**

In [13]:
# 2 - 下载检索的数据。如不设置特定的存储文件夹，TCGAbiolins下载的数据会在工作目录下新建一个名为GDCdata的文件夹用来存储下载的数据文件
GDCdownload(
    query = query,
    method = "api",
    files.per.chunk = 50
    )


GDCdownload will download 88 files. A total of 372.363264 MB




Downloading: 54 MB        

Downloading: 41 MB      

### **c. 读取**

In [14]:
# 3 - 读取数据
# 读取下载的数据，data数据是以SummarizedExperiment class格式组织的，此类数据对象的详细介绍请参阅
dat <- GDCprepare(
    query = query,
    save = TRUE,
    save.filename = paste0("./0-Data/",project,"_mrna.rda"),
    remove.files.prepared = FALSE) 

|====================================================|100%                      Completed after 2 s 


Starting to add information to samples

Adding description to TARGET samples

 => Add clinical information to samples

Available assays in SummarizedExperiment : 
  => unstranded
  => stranded_first
  => stranded_second
  => tpm_unstrand
  => fpkm_unstrand
  => fpkm_uq_unstrand

=> Saving file: ./0-Data/TARGET-OS_mrna.rda

=> File saved



## **批量下载**

In [8]:
projects = TCGAbiolinks:::getGDCprojects()$project_id
TCGAs = grep("TCGA", projects, value = T)
sort(TCGAs)

[1] "TCGA-ACC"  "TCGA-BLCA" "TCGA-BRCA" "TCGA-CESC" "TCGA-CHOL" "TCGA-COAD"
 [7] "TCGA-DLBC" "TCGA-ESCA" "TCGA-GBM"  "TCGA-HNSC" "TCGA-KICH" "TCGA-KIRC"
[13] "TCGA-KIRP" "TCGA-LAML" "TCGA-LGG"  "TCGA-LIHC" "TCGA-LUAD" "TCGA-LUSC"
[19] "TCGA-MESO" "TCGA-OV"   "TCGA-PAAD" "TCGA-PCPG" "TCGA-PRAD" "TCGA-READ"
[25] "TCGA-SARC" "TCGA-SKCM" "TCGA-STAD" "TCGA-TGCT" "TCGA-THCA" "TCGA-THYM"
[31] "TCGA-UCEC" "TCGA-UCS"  "TCGA-UVM"

In [ ]:
## 批量下载RNA-seq数据
for (project in TCGAs) {
    # 1 - 检索需要下载的数据。
    query <- GDCquery(
        project = project,
        data.category = "Transcriptome Profiling",
        data.type = "Gene Expression Quantification",
        workflow.type = "STAR - Counts"
        )
    # 2 - 下载检索的数据。
    GDCdownload(query = query, method = "api", files.per.chunk = 50)
    # 3 - 读取数据
    dat <- GDCprepare(
        query = query,
        save = TRUE,
        save.filename = paste0("./0-Data/",project,"_mrna.rda"),
        remove.files.prepared = FALSE
        ) 
}

In [ ]:
## 批量下载clinical数据
for (project in TCGAs) {
    # 1 - 检索需要下载的数据。
    query <- GDCquery(
        project = project,
        data.category = "Clinical",
        data.type = "Clinical Supplement",
        data.format = "BCR Biotab"
        )
    # 2 - 下载检索的数据。
    GDCdownload(query = query, method = "api", files.per.chunk = 50)
    # 3 - 读取数据
    dat <- GDCprepare(
        query = query, 
        save = TRUE, 
        save.filename = paste0("./0-Data/",project,"_clinical.rda"), 
        remove.files.prepared = FALSE
        ) 
}

In [44]:
grep("clinical_", names(clinical), value = T)
clinical_patient = as.data.frame(clinical$clinical_patient_sarc)

[1] "clinical_follow_up_v4.0_sarc"     "clinical_omf_v4.0_sarc"          
[3] "clinical_nte_sarc"                "clinical_patient_sarc"           
[5] "clinical_follow_up_v4.0_nte_sarc" "clinical_drug_sarc"              
[7] "clinical_radiation_sarc"

In [ ]:
## 批量下载体细胞突变数据
for (project in TCGAs){
  query <- GDCquery(
    project = project, 
    data.category = "Simple Nucleotide Variation", 
    access = "open", 
    data.type = "Masked Somatic Mutation", 
    workflow.type = "Aliquot Ensemble Somatic Variant Merging and Masking"
  )
  GDCdownload(query = query, method = "api", files.per.chunk = 50)
  #读取所有文件
  all_maf <- list.files(path = paste0("./GDCdata/",project,"/Simple_Nucleotide_Variation/Masked_Somatic_Mutation/"),
                        pattern = "ensemble_masked.maf.gz",
                        full.names = T,
                        recursive = T)                   
  maf <- lapply(all_maf, read_tsv, skip = 7)
  #合并数据
  maf_merge <- do.call(rbind, maf)
  save(maf_merge, file = paste0("./0-Data/",project,"_maf.rda"))
}

In [ ]:
## 批量下载甲基化数据
for (project in TCGAs) {
    # 1 - 检索需要下载的数据。
    query <- GDCquery(
        project = project,
        data.category = "DNA Methylation",
        data.type = "Methylation Beta Value",
        platform = "Illumina Human Methylation 450"
        )
    # 2 - 下载检索的数据。
    GDCdownload(query = query, method = "api", files.per.chunk = 50)
    # 3 - 读取数据
    dat <- GDCprepare(
        query = query, 
        save = TRUE, 
        save.filename = paste0("./0-Data/",project,"_methy.rda"), 
        remove.files.prepared = FALSE
        ) 
}

In [ ]:
## 批量下载CNV数据
for (project in TCGAs) {
    # 1 - 检索需要下载的数据。
    query <- GDCquery(
        project = project,
        data.category = "Copy Number Variation",
        data.type="Masked Copy Number Segment"
        )
    # 2 - 下载检索的数据。
    GDCdownload(query = query, method = "api", files.per.chunk = 50)
    # 3 - 读取数据
    dat <- GDCprepare(
        query = query, 
        save = TRUE, 
        save.filename = paste0("./0-Data/",project,"_cnv.rda"), 
        remove.files.prepared = FALSE
        ) 
}

In [ ]:
## 批量下载miRNA数据
for (project in TCGAs) {
    # 1 - 检索需要下载的数据。
    query <- GDCquery(
        project = project,
        data.category = "Transcriptome Profiling",
        data.type = "miRNA Expression Quantification",
        workflow.type = "BCGSC miRNA Profiling"
        )
    # 2 - 下载检索的数据。
    GDCdownload(query = query, method = "api", files.per.chunk = 50)
    # 3 - 读取数据
    dat <- GDCprepare(
        query = query, 
        save = TRUE, 
        save.filename = paste0("./0-Data/",project,"_mirna.rda"), 
        remove.files.prepared = FALSE
        ) 
}

In [ ]:
## 批量下载蛋白质数据
for (project in TCGAs) {
    # 1 - 检索需要下载的数据。
    query <- GDCquery(
        project = project,
        data.category = "Proteome Profiling",
        data.type = "Protein Expression Quantification"
        )
    # 2 - 下载检索的数据。
    GDCdownload(query = query, method = "api", files.per.chunk = 50)
    # 3 - 读取数据
    dat <- GDCprepare(
        query = query, 
        save = TRUE, 
        save.filename = paste0("./0-Data/",project,"_protein.rda"), 
        remove.files.prepared = FALSE
        ) 
}

In [5]:
## 获取病人的肿瘤分型
#PanCancerAtlas_subtypes() The columns “Subtype_Selected” was selected as most prominent subtype classification 
subtypes <- PanCancerAtlas_subtypes()
dim(subtypes)

[1] 7734   10

In [6]:
table(subtypes$cancer.type)


 ACC  AML BLCA BRCA COAD ESCA  GBM HNSC KICH KIRC KIRP  LGG LIHC LUAD LUSC OVCA 
  91  187  129 1218  341  169  606  279   66  442  161  516  196  230  178  489 
PCPG PRAD READ SKCM STAD THCA UCEC  UCS 
 178  333  118  333  383  496  538   57 

In [7]:
head(as.data.frame(subtypes))

,pan.samplesID,cancer.type,Subtype_mRNA,Subtype_DNAmeth,Subtype_protein,Subtype_miRNA,Subtype_CNA,Subtype_Integrative,Subtype_other,Subtype_Selected
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,TCGA-OR-A5J1,ACC,steroid-phenotype-high+proliferation,CIMP-high,NA,miRNA_1,Quiet,COC3,C1A,ACC.CIMP-high
2,TCGA-OR-A5J2,ACC,steroid-phenotype-high+proliferation,CIMP-low,1,miRNA_1,Noisy,COC3,C1A,ACC.CIMP-low
3,TCGA-OR-A5J3,ACC,steroid-phenotype-high,CIMP-intermediate,3,miRNA_6,Chromosomal,COC2,C1A,ACC.CIMP-intermediate
4,TCGA-OR-A5J4,ACC,NA,CIMP-high,NA,miRNA_6,Chromosomal,NA,NA,ACC.CIMP-high
5,TCGA-OR-A5J5,ACC,steroid-phenotype-high,CIMP-intermediate,NA,miRNA_2,Chromosomal,COC2,C1A,ACC.CIMP-intermediate
6,TCGA-OR-A5J6,ACC,steroid-phenotype-low,CIMP-low,2,miRNA_1,Noisy,COC1,C1B,ACC.CIMP-low


In [8]:
save(subtypes,file = "./0-Data/PanCancer_subtypes.rda")

## **TMB计算**

In [9]:
rm(list=ls())
library(TCGAbiolinks)
library(dplyr)
library(stringr)
projects <- getGDCprojects()$project_id
projects <- projects[grepl('^TCGA', projects, perl=TRUE)]
projects <- projects[order(projects)]
#projects=projects[-19]

In [ ]:
pan_TMB=list()
for (project in projects){
  query <- GDCquery(
    project = project, 
    data.category = "Simple Nucleotide Variation", 
    access = "open", 
    data.type = "Masked Somatic Mutation", 
    workflow.type = "Aliquot Ensemble Somatic Variant Merging and Masking"
  )
  GDCdownload(query)
  maf <- GDCprepare(query)
  
  ## TMB calculation
  # Code Reference https://zhuanlan.zhihu.com/p/394609586
  get_TMB <- function(file) {
    use_cols <- c("Hugo_Symbol", "Variant_Classification", "Tumor_Sample_Barcode", 
                    "HGVSc", "t_depth", "t_alt_count")
    # 删除这些突变类型
    mut_type <- c("5'UTR", "3'UTR", "3'Flank", "5'Flank", "Intron", "IGR",
                    "Silent", "RNA", "Splice_Region")
    # 读取文件
    df <- select(file, use_cols)
    data <- df %>% subset(!Variant_Classification %in% mut_type) %>%
    # 计算 VAF
      mutate(vaf = t_alt_count / t_depth) %>%
      group_by(Tumor_Sample_Barcode) %>%
      summarise(mut_num = n(), TMB = mut_num / 30, MaxVAF = max(vaf))
    return(data)
    }
  
  pan_TMB[[project]]=get_TMB(maf)
}

In [ ]:
# For TCGA-MESO
query <- GDCquery(
  project = "TCGA-MESO", 
  data.category = "Simple Nucleotide Variation", 
  access = "open", 
  legacy = FALSE, 
  data.type = "Masked Somatic Mutation", 
  workflow.type = "Aliquot Ensemble Somatic Variant Merging and Masking"
)
GDCdownload(query)

# Reference:https://github.com/PoisonAlien/maftools/issues/838
mafFilePath2 = dir(path = "GDCdata/TCGA-MESO",pattern = "masked.maf.gz$",full.names = T,recursive=T) 
TCGA_MESO = lapply(mafFilePath2, data.table::fread, skip = "Hugo_Symbol")
TCGA_MESO = data.table::rbindlist(l = TCGA_MESO, use.names = TRUE, fill = TRUE)
pan_TMB[["TCGA-MESO"]]=get_TMB(TCGA_MESO)

In [8]:
pan_TMB=do.call(rbind,pan_TMB)
pan_TMB$Cancer=str_split(rownames(pan_TMB),'-',simplify = T)[,2]
pan_TMB$Cancer=str_split(pan_TMB$Cancer,'[.]',simplify = T)[,1]
pan_TMB$Tumor_Sample_Barcode=str_sub(pan_TMB$Tumor_Sample_Barcode,1,16)
pan_TMB=pan_TMB[,c(5,1,3)]
colnames(pan_TMB)=c("Cancer","ID","TMB")
save(pan_TMB,file="./0-Data/PanCancer_TMB.Rdata")

In [9]:
head(pan_TMB)

Cancer,ID,TMB
<chr>,<chr>,<dbl>
BRCA,TCGA-3C-AAAU-01A,0.5333333
BRCA,TCGA-3C-AALI-01A,13.6333333
BRCA,TCGA-3C-AALJ-01A,0.8333333
BRCA,TCGA-3C-AALK-01A,1.3666667
BRCA,TCGA-4H-AAAK-01A,0.5333333
BRCA,TCGA-5L-AAT0-01A,3.9000000
